In [ ]:
# ----- Importer bibliothèques -----

# Bibliothèques vision
import numpy as np
import cv2
import skimage.morphology as morph
import colorsys

# Changement de repère
import math

In [ ]:
# ----- Définir constantes -----

# Définition de la mire
CHECKERBOARD = (6,8)
squareDim = (25, 25)

# Définition des fonctions à utiliser

## Fonctions pour le changement de repère

In [ ]:
# Definition de la matrice de changement de repère a partir des points mesurés
def MatriceChangementRepere(posOrigine, posCoinX, posCoinY):

    nouvX = posCoinX-posOrigine
    nouvY = posCoinY-posOrigine
    
    nouvX[2] = 0
    nouvY[2] = 0
    
    nouvX = nouvX/(math.sqrt(nouvX[0]*nouvX[0]+nouvX[1]*nouvX[1]))
    nouvY = nouvY/(math.sqrt(nouvY[0]*nouvY[0]+nouvY[1]*nouvY[1]))
    #nouvZ = nouvZ/(sqrt(nouvZ[0]^2+nouvZ[1]^2+nouvZ[2]^2))
        
    M = [[nouvX[0], nouvY[0], 0, posOrigine[0]],[nouvX[1], nouvY[1], 0, posOrigine[1]],[0, 0, 1, posOrigine[2]],[0, 0, 0, 1]]
    M_inv = np.linalg.inv(M)
    
    return M_inv

# Effectuer un changement de repère
def ChangementRepere(x, M):
    nx = np.matmul(M_inv, x)
    return nx

## Fonctions pour la vision

In [ ]:
def getLcam(img, CHECKERBOARD, objp, mtx, dist):
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
       
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret == True:
    
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)
    
    Lcam=mtx.dot(np.hstack((cv2.Rodrigues(rvecs)[0],tvecs)))
    return Lcam
    
    
def getOrigin(img, CHECKERBOARD, objp, mtx, dist):
    
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret == True:
    
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        ret, rvecs, tvecs = cv2.solvePnP(objp, corners2, mtx, dist)
    
    # On affiche l'origine sur l'image avec un point rouge
    imgpt, jac = cv2.projectPoints((0,0,0), rvecs, tvecs, mtx, dist)
    return (int(imgpt[0,0,0]), int(imgpt[0,0,1]))
    
    
def realPosition(px, py, Z, Lcam, squareDim):

    X=np.linalg.inv(np.hstack((Lcam[:,0:2],np.array([[-1*px],[-1*py],[-1]])))).dot((-Z*Lcam[:,2]-Lcam[:,3]))
    
    return (X[0]*squareDim[0], X[1]*squareDim[1])


# Fonction combinant blur, opening et closing sur la composante de saturation (HSV)
def advancedThreshold(img, blur, th, closingTh, openingTh):

    imgBlur = cv2.GaussianBlur(img, (blur,blur), 7)
    imgHSV = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2HSV)
    ret, imgTh = cv2.threshold(imgHSV[:,:,1], th, 255, cv2.THRESH_BINARY)
    imgOpen = cv2.morphologyEx(imgTh, cv2.MORPH_OPEN, morph.disk(openingTh))
    imgClos = cv2.morphologyEx(imgOpen, cv2.MORPH_CLOSE, morph.disk(closingTh))
    
    return imgClos

            
def getContoursNoDisplay(img,imgContour,th,Lcam,squareDim):
    
    # Recupere les contours
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
      
    # Compte le nombre d'objets
    objectCount = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # On ne considere que les contours suffisement grands
        if area > th:
            
            objectCount = objectCount + 1
            
    objectPositions = np.zeros([int(objectCount),3])
    
    # Recuperation des positions
    i = 0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        # On ne considere que les contours suffisement grands
        if area > th:
            
            # Dessiner les contours
            #cv2.drawContours(imgContour, cnt, -1, (255, 0, 255), 7)
            
            # Polygone a partir du contour
            #peri = cv2.arcLength(cnt, True)
            #approx = cv2.approxPolyDP(cnt, 0.02*peri, True)
            
            # Trouver le centre du polygone et recuperer sa couleur
            M = cv2.moments(cnt)
            cX = int(M["m10"]/M["m00"])
            cY = int(M["m01"]/M["m00"])
            #r = int(imgContour[cY, cX, 0])
            #g = int(imgContour[cY, cX, 1])
            #b = int(imgContour[cY, cX, 2])           
            #cv2.circle(imgContour, (cX, cY), 5, (0,0,0), -1)
            pos = realPosition(cX, cY, 0, Lcam, squareDim)
            
            objectPositions[i, 0] = pos[0]
            objectPositions[i, 1] = pos[1]
            i = i+1     
            # Encadrement des polygones
            #x, y, w, h = cv2.boundingRect(approx)
            #cv2.rectangle(imgContour, (x, y), (x+w, y+h), (r, g, b), 5)
            
            # Affichage de la surface
            #cv2.putText(imgContour, "Area : "+str(int(area)), (x+20,y+20), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
            #cv2.putText(imgContour, "X : "+str(round(pos[0],2)), (x+20,y+45), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
            #cv2.putText(imgContour, "Y : "+str(round(pos[1],2)), (x+20,y+70), cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,255,0), 2)
            
            
    return objectPositions

def CalibrationPosition(imgCalib, CHECKERBOARD, squareDim):
    
    objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
    objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
    
    # Charger paramètres intrinsèques
    with np.load("./calibration/calibration.npz") as X:
        mtx, dist = [X[i] for i in ('mtx','dist')]
    
    Lcam = getLcam(imgCalib, CHECKERBOARD, objp, mtx, dist)
    origin = getOrigin(imgCalib, CHECKERBOARD, objp, mtx, dist)
    
    return [Lcam, origin]

def GetObjectsPositions(img, opening, closing, blur, threshold, minSize, Lcam, origin, CHECKERBOARD, squareDim):
    
    imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Filtrage pour utilisation
    imgFiltered = advancedThreshold(img, blur*2+1, threshold, closing*2+1, opening*2+1)    # IMPORTER CA

    # Detection du contour
    imgCanny = cv2.Canny(imgFiltered, 4, 4)


    imgContour = img.copy()
    objectsPositions = getContoursNoDisplay(imgCanny, imgContour,minSize,Lcam,squareDim)    # IMPORTER CA

        
    return objectsPositions

# Corps du programme

In [ ]:
#Constantes
positionDepot = [1, 1, 1] # a definir
hauteurStandard = 1
offsetSaisie = 1



# se mettre en position 0 (lent)

# attendre input

# se déplacer en position de capture d'image (lent)
# attendre la stabilisation
[Lcam, origin] = CalibrationPosition(imgCalib, CHECKERBOARD, squareDim)

# attendre input (il faudra mettre les jenda avant de faire l'input)
# ouvrir pince
objectsPositions = GetObjectsPositions(img, 5, 15, 2, 147, 3000, Lcam, origin, CHECKERBOARD, squareDim)





M = MatriceChangementRepere(posOrigine, posCoinX, posCoinY)


for i in range(0, np.shape(objectsPositions)[0]):
    
    X_Rcam = np.array[objectsPositions[0], objectsPositions[1], 0]
    X_Rrobot = ChangementRepere(x, M)
    X_Rrobot[2] = hauteurStandard+offsetSaisie
    
    # Go to X_Rrobot (rapide)
    
    X_Rrobot[2] = hauteurStandard
    
    # Go to X_Rrobot (lent)
    # attendre stabilisation
    # Fermer pince
    
    X_Rrobot[2] = hauteurStandard+offsetSaisie
    
    # Go to X_Rrobot (lent)
    
    # Go to position depot (rapide)
    # attendre stabilisation
    #ouvrir pince





